In [167]:
# Lib & Dependencies

import pandas as pd
import numpy as np
import mlflow
import os
import scipy as sp
import gc
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.metrics import accuracy_score, f1_score, auc
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ridge_regression, RidgeCV, LassoCV, BayesianRidge, LinearRegression
from sklearn.preprocessing import LabelEncoder

In [160]:
!ls datasets

diabetes_012_health_indicators_BRFSS2015.csv.zip
diabetes_binary_5050split_health_indicators_BRFSS2015.csv.zip


In [161]:
# df2 = pd.read_csv("datasets/diabetes_012_health_indicators_BRFSS2015.csv.zip")
df = pd.read_csv("datasets/diabetes_binary_5050split_health_indicators_BRFSS2015.csv.zip")

In [162]:
df.shape

(70692, 22)

In [163]:
df.rename(columns={"Diabetes_012":"Diabetes_binary"}, inplace=True)

In [164]:
train = df.drop('Diabetes_binary', axis=1)
targets = df.loc[:, "Diabetes_binary"]

In [165]:
train.shape, targets.shape

((70692, 21), (70692,))

In [166]:
!ls

README.md          diabetes_mlflow.db monitoring         web_service
datasets           mlruns             ochestration
diabetes.ipynb     models             requirements.txt


train

## Experiment

In [171]:
os.environ["AWS_PROFILE"] = "hakymulla" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-3-94-180-117.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [172]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-3-94-180-117.compute-1.amazonaws.com:5000'


In [173]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://diabetes-mlflow-artifact/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://diabetes-mlflow-artifact/1', experiment_id='1', lifecycle_stage='active', name='diabetes_experiment', tags={}>,
 <Experiment: artifact_location='s3://diabetes-mlflow-artifact/3', experiment_id='3', lifecycle_stage='active', name='diabetes_', tags={}>]

In [174]:
mlflow.set_experiment("diabetes_experiment_main")
# # mlflow.set_tracking_uri("http://127.0.0.1:5000")
# mlflow.set_tracking_uri("sqlite:///diabetes_mlflow.db")

2022/08/05 07:31:12 INFO mlflow.tracking.fluent: Experiment with name 'diabetes_experiment_main' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://diabetes-mlflow-artifact/4', experiment_id='4', lifecycle_stage='active', name='diabetes_experiment_main', tags={}>

In [175]:
models = {"rf":RandomForestClassifier(n_jobs=-1), "lgbm": LGBMClassifier(n_jobs=-1),
            "xgb": XGBClassifier(n_jobs=-1), "cb": CatBoostClassifier(silent=True)}

In [176]:
RANDOM_STATE = 1111

In [177]:
def kfold_lightgbm(df, target_name,  num_folds, model, debug= False, params=None):
    with mlflow.start_run():
        FOLDS = num_folds

        mlflow.set_tag("Developer", "Hakymulla")

        mlflow.log_param("Train Data", "datasets/diabetes_binary_5050split_health_indicators_BRFSS2015.csv.zip")

        mlflow.log_param("RANDOM_STATE", RANDOM_STATE)
        mlflow.log_param("FOLDS", FOLDS)
        mlflow.log_param("Models", str(models[model]))


        train_df = df.drop(target_name, axis=1)
        target = df.loc[:, target_name]
        print("Starting Training. Train shape: {}".format(train_df.shape))
        gc.collect()

    #     Cross validation model
        folds = KFold(n_splits= FOLDS, shuffle=True, random_state=RANDOM_STATE)
        
        oof_preds = np.zeros(train_df.shape[0])
        for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, target)):    
            train_x, train_y = train_df.iloc[train_idx], target.iloc[train_idx]
            valid_x, valid_y = train_df.iloc[valid_idx], target.iloc[valid_idx]

            clf = models[model]
            
            clf.fit(train_x, train_y)
            # clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], early_stopping_rounds= 100)
            oof_preds[valid_idx] = clf.predict(valid_x)
        

            score = f1_score(valid_y, oof_preds[valid_idx])
            accuracy = accuracy_score(valid_y, oof_preds[valid_idx])
            print(f'---------> Fold {n_fold + 1} {score}')

            del clf, train_x, train_y, valid_x, valid_y
            gc.collect()

        score = f1_score(target, oof_preds)
        full_accuracy_score = accuracy_score(target, oof_preds)
        mlflow.log_metric("f1_score", score)
        mlflow.log_metric("accuracy", full_accuracy_score)
        print('Full F1 Score score %.8f' % score)

In [178]:
kfold_lightgbm(df=df, target_name='Diabetes_binary', num_folds=3, model="rf")

Starting Training. Train shape: (70692, 21)
---------> Fold 1 0.7447289466113189
---------> Fold 2 0.7495925684485005
---------> Fold 3 0.7455495672056183
Full F1 Score score 0.74662996


In [180]:
# kfold_lightgbm(df=df, target_name='Diabetes_binary', num_folds=3, model="xgb")

In [181]:
kfold_lightgbm(df=df, target_name='Diabetes_binary', num_folds=3, model="lgbm")

Starting Training. Train shape: (70692, 21)
---------> Fold 1 0.7618814506588341
---------> Fold 2 0.7644422913891243
---------> Fold 3 0.7624954396205764
Full F1 Score score 0.76294403


In [182]:
kfold_lightgbm(df=df, target_name='Diabetes_binary', num_folds=3, model="cb")

Starting Training. Train shape: (70692, 21)
---------> Fold 1 0.7643868020718626
---------> Fold 2 0.7616653829002309
---------> Fold 3 0.7624959428756898
Full F1 Score score 0.76284639


## HyperParameter Tunning

In [183]:
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [184]:
RANDOM_STATE = 1111
FOLDS = 5

In [186]:
# frac = int(df.shape[0] *0.8)

In [23]:
# df_train = df.loc[:frac, ]
# X_train = df_train.drop("Diabetes_binary", axis=1)
# y_train = df_train.loc[:, 'Diabetes_binary']

# df_valid = df.loc[frac:, ]
# X_val = df_valid.drop("Diabetes_binary", axis=1)
# y_val = df_valid.loc[:, 'Diabetes_binary']

# X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [24]:
# train = lgb.Dataset(X_train, label=y_train)
# valid = lgb.Dataset(X_val, label=y_val)

In [25]:
# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag("model", "xgboost")
#         mlflow.log_param("RANDOM_STATE", RANDOM_STATE)
#         mlflow.log_param("FOLDS", FOLDS)
#         mlflow.log_param("Models", "LightGBM")
#         mlflow.log_params(params)
#         booster = lgb.train(
#             params=params,
#             train_set=train,
#             # num_boost_round=1000,
#             num_boost_round=10,
#             valid_sets=valid,
#             early_stopping_rounds=50
#         )
#         y_pred = booster.predict(X_val, raw_score=False)
#         y_pred = np.round(y_pred)

#         score = f1_score(y_val, y_pred)
#         mlflow.log_metric("score", score)

#     return {'loss': -score, 'status': STATUS_OK}

In [187]:
def objective(params):
    with mlflow.start_run():

        mlflow.set_tag("Developer", "Hakymulla")
        mlflow.log_param("Train Data", "datasets/diabetes_binary_5050split_health_indicators_BRFSS2015.csv.zip")
        mlflow.log_param("RANDOM_STATE", RANDOM_STATE)
        mlflow.log_param("FOLDS", FOLDS)
        mlflow.log_params(params)
        mlflow.log_param("Models", "LightGBM")

        train_df = df.drop('Diabetes_binary', axis=1)
        target = df.loc[:, 'Diabetes_binary']

        print("Starting Training. Train shape: {}".format(train_df.shape))
        gc.collect()

    #     Cross validation model
        folds = KFold(n_splits= FOLDS, shuffle=True, random_state=RANDOM_STATE)
        
        oof_preds = np.zeros(train_df.shape[0])
        for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, target)):    
            train_x, train_y = train_df.iloc[train_idx], target.iloc[train_idx]
            valid_x, valid_y = train_df.iloc[valid_idx], target.iloc[valid_idx]

            train = lgb.Dataset(train_x, label=train_y)
            valid = lgb.Dataset(valid_x, label=valid_y)

            booster = lgb.train(
            params=params,
            train_set=train,
            num_boost_round=1000,
            valid_sets=valid,
            early_stopping_rounds=50
            )

            y_pred = booster.predict(valid_x, raw_score=False)
            y_pred = np.round(y_pred)
            oof_preds[valid_idx] = y_pred

            f_score = f1_score(valid_y, oof_preds[valid_idx])
            accuracy = accuracy_score(valid_y, oof_preds[valid_idx])

            print(f'---------> Fold {n_fold + 1} {f_score}')

            del booster, train_x, train_y, valid_x, valid_y
            gc.collect()

        full_score = f1_score(target, oof_preds)
        full_accuracy_score = accuracy_score(target, oof_preds)

        mlflow.log_metric("f1_score", full_score)
        mlflow.log_metric("accuracy", full_accuracy_score)

    return {'loss': -full_score, 'status': STATUS_OK}

In [188]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    # 'objective': 'binary',
    'seed': 42,
    'verbose':-1
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials(),
    verbose=False
)

Starting Training. Train shape: (70692, 21)
[1]	valid_0's l2: 0.229089
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.213844
[3]	valid_0's l2: 0.20258
[4]	valid_0's l2: 0.194169
[5]	valid_0's l2: 0.188063
[6]	valid_0's l2: 0.183418
[7]	valid_0's l2: 0.179659
[8]	valid_0's l2: 0.176985
[9]	valid_0's l2: 0.174923
[10]	valid_0's l2: 0.173062
[11]	valid_0's l2: 0.171549
[12]	valid_0's l2: 0.170455
[13]	valid_0's l2: 0.169525
[14]	valid_0's l2: 0.168863
[15]	valid_0's l2: 0.168295
[16]	valid_0's l2: 0.167808
[17]	valid_0's l2: 0.167466
[18]	valid_0's l2: 0.167217
[19]	valid_0's l2: 0.167002
[20]	valid_0's l2: 0.166804
[21]	valid_0's l2: 0.166672
[22]	valid_0's l2: 0.166527
[23]	valid_0's l2: 0.166485
[24]	valid_0's l2: 0.166396
[25]	valid_0's l2: 0.166321
[26]	valid_0's l2: 0.166266
[27]	valid_0's l2: 0.166199
[28]	valid_0's l2: 0.166199
[29]	valid_0's l2: 0.166166
[30]	valid_0's l2: 0.16622
[31]	valid_0's l2: 0.166208
[32]	valid_0's l2: 0.166198
[33]	valid

/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[61]	valid_0's l2: 0.166237
[62]	valid_0's l2: 0.166248
[63]	valid_0's l2: 0.166272
[64]	valid_0's l2: 0.166285
[65]	valid_0's l2: 0.166303
[66]	valid_0's l2: 0.16631
[67]	valid_0's l2: 0.166356
[68]	valid_0's l2: 0.166359
[69]	valid_0's l2: 0.166352
[70]	valid_0's l2: 0.16638
[71]	valid_0's l2: 0.166402
[72]	valid_0's l2: 0.166394
[73]	valid_0's l2: 0.166385
[74]	valid_0's l2: 0.166378
[75]	valid_0's l2: 0.166345
[76]	valid_0's l2: 0.166347
[77]	valid_0's l2: 0.166366
[78]	valid_0's l2: 0.166393
[79]	valid_0's l2: 0.166389
[80]	valid_0's l2: 0.166381
[81]	valid_0's l2: 0.166391
[82]	valid_0's l2: 0.166395
[83]	valid_0's l2: 0.166396
[84]	valid_0's l2: 0.166398
Early stopping, best iteration is:
[34]	valid_0's l2: 0.16615
---------> Fold 1 0.75983736475777


/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.229493
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.214656
[3]	valid_0's l2: 0.203636
[4]	valid_0's l2: 0.195557
[5]	valid_0's l2: 0.189362
[6]	valid_0's l2: 0.184844
[7]	valid_0's l2: 0.181345
[8]	valid_0's l2: 0.178564
[9]	valid_0's l2: 0.176393
[10]	valid_0's l2: 0.174853
[11]	valid_0's l2: 0.173489
[12]	valid_0's l2: 0.172464
[13]	valid_0's l2: 0.171631
[14]	valid_0's l2: 0.170969
[15]	valid_0's l2: 0.170452
[16]	valid_0's l2: 0.170028
[17]	valid_0's l2: 0.169617
[18]	valid_0's l2: 0.169364
[19]	valid_0's l2: 0.16914
[20]	valid_0's l2: 0.169005
[21]	valid_0's l2: 0.168856
[22]	valid_0's l2: 0.16876
[23]	valid_0's l2: 0.168619
[24]	valid_0's l2: 0.168544
[25]	valid_0's l2: 0.168521
[26]	valid_0's l2: 0.168444
[27]	valid_0's l2: 0.168385
[28]	valid_0's l2: 0.168329
[29]	valid_0's l2: 0.16831
[30]	valid_0's l2: 0.168302
[31]	valid_0's l2: 0.168307
[32]	valid_0's l2: 0.168262
[33]	valid_0's l2: 0.168251
[34]	valid_0's l2: 0.168241

...

[9]	valid_0's l2: 0.190395
[10]	valid_0's l2: 0.187671
[11]	valid_0's l2: 0.185306
[12]	valid_0's l2: 0.183171
[13]	valid_0's l2: 0.181334
[14]	valid_0's l2: 0.179753
[15]	valid_0's l2: 0.178296
[16]	valid_0's l2: 0.177043
[17]	valid_0's l2: 0.175916
[18]	valid_0's l2: 0.174897
[19]	valid_0's l2: 0.173935
[20]	valid_0's l2: 0.173188
[21]	valid_0's l2: 0.172451
[22]	valid_0's l2: 0.171782
[23]	valid_0's l2: 0.171192
[24]	valid_0's l2: 0.170655
[25]	valid_0's l2: 0.170206
[26]	valid_0's l2: 0.169871
[27]	valid_0's l2: 0.169475
[28]	valid_0's l2: 0.169155
[29]	valid_0's l2: 0.168851
[30]	valid_0's l2: 0.16856
[31]	valid_0's l2: 0.168301
[32]	valid_0's l2: 0.168073
[33]	valid_0's l2: 0.167863
[34]	valid_0's l2: 0.167725
[35]	valid_0's l2: 0.167543
[36]	valid_0's l2: 0.16744
[37]	valid_0's l2: 0.167324
[38]	valid_0's l2: 0.167197
[39]	valid_0's l2: 0.167052
[40]	valid_0's l2: 0.166979
[41]	valid_0's l2: 0.166897
[42]	valid_0's l2: 0.166804
[43]	valid_0's l2: 0.166749
[44]	valid_0's l2: 0.16

/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[13]	valid_0's l2: 0.180118
[14]	valid_0's l2: 0.178565
[15]	valid_0's l2: 0.177131
[16]	valid_0's l2: 0.17581
[17]	valid_0's l2: 0.174729
[18]	valid_0's l2: 0.173777
[19]	valid_0's l2: 0.172864
[20]	valid_0's l2: 0.172092
[21]	valid_0's l2: 0.171426
[22]	valid_0's l2: 0.170822
[23]	valid_0's l2: 0.170214
[24]	valid_0's l2: 0.169739
[25]	valid_0's l2: 0.169343
[26]	valid_0's l2: 0.168954
[27]	valid_0's l2: 0.168623
[28]	valid_0's l2: 0.168299
[29]	valid_0's l2: 0.168047
[30]	valid_0's l2: 0.167846
[31]	valid_0's l2: 0.167638
[32]	valid_0's l2: 0.167436
[33]	valid_0's l2: 0.167262
[34]	valid_0's l2: 0.167095
[35]	valid_0's l2: 0.166937
[36]	valid_0's l2: 0.166813
[37]	valid_0's l2: 0.166721
[38]	valid_0's l2: 0.166622
[39]	valid_0's l2: 0.166548
[40]	valid_0's l2: 0.166489
[41]	valid_0's l2: 0.166419
[42]	valid_0's l2: 0.166362
[43]	valid_0's l2: 0.166308
[44]	valid_0's l2: 0.166253
[45]	valid_0's l2: 0.166233
[46]	valid_0's l2: 0.166189
[47]	valid_0's l2: 0.166161
[48]	valid_0's l2: 0.

/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.238325
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.228636
[3]	valid_0's l2: 0.220444
[4]	valid_0's l2: 0.21347
[5]	valid_0's l2: 0.20751
[6]	valid_0's l2: 0.202507
[7]	valid_0's l2: 0.198188
[8]	valid_0's l2: 0.19449
[9]	valid_0's l2: 0.191393
[10]	valid_0's l2: 0.188726
[11]	valid_0's l2: 0.18637
[12]	valid_0's l2: 0.184361
[13]	valid_0's l2: 0.182685
[14]	valid_0's l2: 0.18118
[15]	valid_0's l2: 0.179811
[16]	valid_0's l2: 0.178683
[17]	valid_0's l2: 0.177635
[18]	valid_0's l2: 0.17665
[19]	valid_0's l2: 0.175766
[20]	valid_0's l2: 0.175021
[21]	valid_0's l2: 0.17434
[22]	valid_0's l2: 0.173689
[23]	valid_0's l2: 0.173124
[24]	valid_0's l2: 0.172678
[25]	valid_0's l2: 0.17225
[26]	valid_0's l2: 0.171876
[27]	valid_0's l2: 0.171535
[28]	valid_0's l2: 0.171223
[29]	valid_0's l2: 0.170943
[30]	valid_0's l2: 0.170701
[31]	valid_0's l2: 0.170531
[32]	valid_0's l2: 0.170321
[33]	valid_0's l2: 0.170142
[34]	valid_0's l2: 0.169994
[35]

## Run model with the best hyperparamter and save in s3 bucket

In [222]:
with mlflow.start_run():

    best_params = {'learning_rate':'0.10552983694225122',
                'max_depth':'89',
                'min_child_weight':'1.704681566723118',
                'reg_alpha':'0.010202520050703611',
                'reg_lambda':'0.046206444839271325',
                'seed':'42'
                    }

    mlflow.set_experiment("diabetes_experiment_main")
    mlflow.set_tag("Developer", "Hakymulla")
    mlflow.log_param("Train Data", "datasets/diabetes_binary_5050split_health_indicators_BRFSS2015.csv.zip")
    mlflow.log_param("RANDOM_STATE", RANDOM_STATE)
    mlflow.log_param("FOLDS", FOLDS)
    mlflow.log_params(best_params)
    mlflow.log_param("Models", "LightGBM")

    train_df = df.drop('Diabetes_binary', axis=1)
    target = df.loc[:, 'Diabetes_binary']

    print("Starting Training. Train shape: {}".format(train_df.shape))
    gc.collect()

#     Cross validation model
    folds = KFold(n_splits= FOLDS, shuffle=True, random_state=RANDOM_STATE)
    
    oof_preds = np.zeros(train_df.shape[0])
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, target)):    
        train_x, train_y = train_df.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], target.iloc[valid_idx]

        train = lgb.Dataset(train_x, label=train_y)
        valid = lgb.Dataset(valid_x, label=valid_y)

        booster = lgb.train(
        params=best_params,
        train_set=train,
        num_boost_round=1000,
        valid_sets=valid,
        early_stopping_rounds=50
        )

        y_pred = booster.predict(valid_x, raw_score=False)
        y_pred = np.round(y_pred)
        oof_preds[valid_idx] = y_pred

        f_score = f1_score(valid_y, oof_preds[valid_idx])
        accuracy = accuracy_score(valid_y, oof_preds[valid_idx])

        print(f'---------> Fold {n_fold + 1} {f_score}')

        del booster, train_x, train_y, valid_x, valid_y
        gc.collect()

    full_score = f1_score(target, oof_preds)
    full_accuracy_score = accuracy_score(target, oof_preds)

    mlflow.log_metric("f1_score", full_score)
    mlflow.log_metric("accuracy", full_accuracy_score)


    # train = lgb.Dataset(train_df, label=target)
    # booster = lgb.train(
    #     params=best_params,
    #     train_set=train,
    #     num_boost_round=1000
    #     )

    # booster.save_model('lgbm.txt')
    
    mlflow.lightgbm.save_model(booster, path="models")
    mlflow.lightgbm.log_model(booster, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

    

Starting Training. Train shape: (70692, 21)
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003618 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 197
[LightGBM] [Info] Number of data points in the train set: 56553, number of used features: 21
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 0.501034
[1]	valid_0's l2: 0.23519
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.223241
[3]	valid_0's l2: 0.21351
[4]	valid_0's l2: 

/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[64]	valid_0's l2: 0.166108
[65]	valid_0's l2: 0.166107
[66]	valid_0's l2: 0.166132
[67]	valid_0's l2: 0.166142
[68]	valid_0's l2: 0.166139
[69]	valid_0's l2: 0.166144
[70]	valid_0's l2: 0.166146
[71]	valid_0's l2: 0.166154
[72]	valid_0's l2: 0.166151
[73]	valid_0's l2: 0.166189
[74]	valid_0's l2: 0.166199
[75]	valid_0's l2: 0.166175
[76]	valid_0's l2: 0.16617
[77]	valid_0's l2: 0.166137
[78]	valid_0's l2: 0.166125
[79]	valid_0's l2: 0.166139
[80]	valid_0's l2: 0.16612
[81]	valid_0's l2: 0.166148
[82]	valid_0's l2: 0.166134
[83]	valid_0's l2: 0.166135
[84]	valid_0's l2: 0.166135
[85]	valid_0's l2: 0.166145
[86]	valid_0's l2: 0.166142
[87]	valid_0's l2: 0.166129
[88]	valid_0's l2: 0.166142
[89]	valid_0's l2: 0.166129
[90]	valid_0's l2: 0.166128
[91]	valid_0's l2: 0.166121
[92]	valid_0's l2: 0.166105
[93]	valid_0's l2: 0.166125
[94]	valid_0's l2: 0.166118
[95]	valid_0's l2: 0.166122
[96]	valid_0's l2: 0.16613
[97]	valid_0's l2: 0.166141
[98]	valid_0's l2: 0.166131
[99]	valid_0's l2: 0.16

/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 199
[LightGBM] [Info] Number of data points in the train set: 56553, number of used features: 21
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 0.499920
[1]	valid_0's l2: 0.235467
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.223774
[3]	valid_0's l2: 0.214287
[4]	valid_0's l2: 0.206546
[5]	valid_0's l2: 0.200215
[6]	va

...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003436 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 199
[LightGBM] [Info] Number of data points in the train set: 56554, number of used features: 21
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 0.498550
[1]	valid_0's l2: 0.23533
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.223438
[3]	valid_0's l2: 0.213752
[4]	valid_0's l2: 0.205793
[5]	valid_0's l2: 0.199327
[6]	val

/Users/hakymulla/opt/anaconda3/envs/diabetes/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 198
[LightGBM] [Info] Number of data points in the train set: 56554, number of used features: 21
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 0.499982
[1]	valid_0's l2: 0.235458
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.223931
[3]	valid_0's l2: 0.214666
[4]	valid_0's l2: 0.207052
[5]	valid_0's l2: 0.200883
[6]	va

2022/08/05 08:12:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /Users/hakymulla/Documents/Files/MLOps/Diabetes/models, flavor: lightgbm), fall back to return ['lightgbm==3.3.2']. Set logging level to DEBUG to see the full traceback.


default artifacts URI: 's3://diabetes-mlflow-artifact/4/c37e8ebbdaf74dcea8c5ca82afa398ac/artifacts'


In [43]:
logged_model = 'runs:/72f0bf4ee7044c8492a02c6abd7d02c2/models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

### Interacting with the model registry

In [75]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [76]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1658948617289, description='', last_updated_timestamp=1658948618245, latest_versions=[<ModelVersion: creation_timestamp=1658948618245, current_stage='None', description='', last_updated_timestamp=1658948618245, name='lgbm-diabetes-best', run_id='bef6031403d0466db8de93ccad05a4e3', run_link='', source='s3://diabetes-mlflow-artifact/1/bef6031403d0466db8de93ccad05a4e3/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='lgbm-diabetes-best', tags={}>]

In [79]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='lgbm-diabetes-best'
)

Registered model 'lgbm-diabetes-best' already exists. Creating a new version of this model...
2022/07/27 20:15:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: lgbm-diabetes-best, version 2
Created version '2' of model 'lgbm-diabetes-best'.


<ModelVersion: creation_timestamp=1658949326980, current_stage='None', description='', last_updated_timestamp=1658949326980, name='lgbm-diabetes-best', run_id='b0ad677e1abb4a579074553839ece302', run_link='', source='s3://diabetes-mlflow-artifact/1/b0ad677e1abb4a579074553839ece302/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>

In [97]:
model = LGBMClassifier(n_jobs=-1)
model.fit(train, targets)

LGBMClassifier()

In [98]:
model.booster_.save_model('mode.txt')

In [190]:
import lightgbm as lgb

In [203]:
RUN_ID = 'ca9aae4bb458470586c23ca39e7afc86'

In [209]:
logged_model = f's3://diabetes-mlflow-artifact/4/{RUN_ID}/artifacts/models/'

In [207]:
logged_model = 's3://diabetes-mlflow-artifact/4/ca9aae4bb458470586c23ca39e7afc86/artifacts/models/'

In [210]:
model = mlflow.lightgbm.load_model(logged_model)

In [211]:
# bst = lgb.Booster(model_file='~/Downloads/model.lgb')

In [217]:
test_df = df.drop('Diabetes_binary', axis=1).head()

In [218]:
model.predict(test_df)

array([0.12592094, 0.5137865 , 0.18593124, 0.71732981, 0.18620113])